In [529]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from functools import reduce
from datetime import datetime, timedelta

In [5]:
train_b = pd.read_excel('Train/train_bureau.xlsx')

In [6]:
train_d = pd.read_excel('Train/train_Data.xlsx')

In [7]:
test_b = pd.read_excel('Test/test_bureau.xlsx')

In [8]:
test_d = pd.read_excel('Test/test_Data.xlsx')

In [256]:
train_bb = train_b.copy()

In [257]:
test_bb = test_b.copy()

In [91]:
# Data Treatment:
#     -categorical vars

In [ ]:
# id -dont do it
# self indiactor -nochange
# match type- not req
# acct type -required 3 columns chosen for remaning crosstab then combine cols
    -
 #contributor type ARC is not ptesent in test so crosstab and then delete arc   
# ownership indicator no rpoblem
# account status not present in test Sold/Purchased ,Cancelled cross tab then delete                            
# credit limit CREDIT-LIMIT/SANC AMT  DISBURSED-AMT/HIGH CREDIT kbindiscretization
# installment not present in test data F04 F07  F08 crosstab then delete it
# asset class 2 not ptresent in test crosstab and remove it

In [127]:
train_SELF_INDICATOR=pd.crosstab(train.ID,train['SELF-INDICATOR'],dropna= True).reset_index()
test_SELF_INDICATOR=pd.crosstab(test.ID,test['SELF-INDICATOR'],dropna= True).reset_index()

In [129]:
train_MATCH_TYPE=pd.crosstab(train.ID,train['MATCH-TYPE'],dropna= True).reset_index()
test_MATCH_TYPE=pd.crosstab(test.ID,test['MATCH-TYPE'],dropna= True).reset_index()

In [172]:
x = set(train_bb['ACCT-TYPE'].unique())  
y = set(test_bb['ACCT-TYPE'].unique()) 
common = x.intersection(y)
print(list(common)) #now decide cols to keep 

['Property Loan', 'SHG Individual', 'Commercial Vehicle Loan', 'Construction Equipment Loan', 'Loan to Professional', 'Overdraft', 'Microfinance Business Loan', 'Business Non-Funded Credit Facility-Priority Sector-Agriculture', 'Two-Wheeler Loan', 'Business Loan General', 'Personal Loan', 'Loan Against Shares / Securities', 'Microfinance Personal Loan', 'Consumer Loan', 'Tractor Loan', 'Business Loan Unsecured', 'Prime Minister Jaan Dhan Yojana - Overdraft', 'Business Non-Funded Credit Facility General', 'Business Loan - Secured', 'Secured Credit Card', 'Auto Loan (Personal)', 'Gold Loan', 'Pradhan Mantri Awas Yojana - CLSS', 'Individual', 'Business Loan Against Bank Deposits', 'Microfinance Others', 'Mudra Loans   Shishu / Kishor / Tarun', 'Non-Funded Credit Facility', 'Business Loan Priority Sector  Others', 'Business Loan Priority Sector  Agriculture', 'Used Car Loan', 'Credit Card', 'Commercial Equipment Loan', 'Kisan Credit Card', 'Other', 'Loan Against Bank Deposits', 'Microfinan

In [152]:
train_ACCT_TYPE=pd.crosstab(train.ID,train['ACCT-TYPE'],dropna= True).reset_index()
test_ACCT_TYPE=pd.crosstab(test.ID,test['ACCT-TYPE'],dropna= True).reset_index()

cols_to_keep =['Tractor Loan','Gold Loan','Business Loan Priority Sector  Agriculture']

train_ACCT_TYPE["Others"] = pd.DataFrame(train_ACCT_TYPE.drop(columns=cols_to_keep).sum(axis=1))
test_ACCT_TYPE["Others"] = pd.DataFrame(test_ACCT_TYPE.drop(columns=cols_to_keep).sum(axis=1))
filter_cols =cols_to_keep + ['Others'] 
train_ACCT_TYPE = train_ACCT_TYPE.filter(filter_cols)
test_ACCT_TYPE = test_ACCT_TYPE.filter(filter_cols)

In [175]:
train_CONTRIBUTOR_TYPE=pd.crosstab(train.ID,train['CONTRIBUTOR-TYPE'],dropna= True).drop(columns = ['ARC']).reset_index()
test_CONTRIBUTOR_TYPE=pd.crosstab(test.ID,test['CONTRIBUTOR-TYPE'],dropna= True).reset_index()


In [178]:
train_OWNERSHIP_IND=pd.crosstab(train.ID,train['OWNERSHIP-IND'],dropna= True).reset_index()
test_OWNERSHIP_IND=pd.crosstab(test.ID,test['OWNERSHIP-IND'],dropna= True).reset_index()


In [181]:
train_ACCOUNT_STATUS=pd.crosstab(train.ID,train['ACCOUNT-STATUS'],dropna= True).drop(columns=['Sold/Purchased','Cancelled']).reset_index()
test_ACCOUNT_STATUS=pd.crosstab(test.ID,test['ACCOUNT-STATUS'],dropna= True).reset_index()


In [183]:
train_INSTALLMENT_FREQUENCY=pd.crosstab(train.ID,train['INSTALLMENT-FREQUENCY'],dropna= True).drop(columns=['F04','F07','F08']).reset_index()
test_INSTALLMENT_FREQUENCY=pd.crosstab(test.ID,test['INSTALLMENT-FREQUENCY'],dropna= True).reset_index()

In [184]:
train_ASSET_CLASS=pd.crosstab(train.ID,train['ASSET_CLASS'],dropna= True).drop(columns=['2']).reset_index()
test_ASSET_CLASS=pd.crosstab(test.ID,test['ASSET_CLASS'],dropna= True).reset_index()

In [376]:
def onehotencoder(train,test,cols=[]):
    
    from sklearn.preprocessing import OneHotEncoder
    #object_cols = catVar1(train) #catVar1 gives desired categorical column and not all object columns
    object_cols=cols
    print(object_cols)
    
    OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train[object_cols]))
    OH_cols_test = pd.DataFrame(OH_encoder.transform(test[object_cols]))

    # One-hot encoding removed index; put it back
    OH_cols_train.index = train.index
    OH_cols_test.index = test.index

    ##hack for restoring columns names just like get dummies
    column_name = OH_encoder.get_feature_names(object_cols)
    OH_cols_train.columns = column_name
    OH_cols_test.columns = column_name
    

    # Remove desired categorical columns (will replace with one-hot encoding)
    num_train = train.drop(object_cols, axis=1)
    num_test = test.drop(object_cols, axis=1)

    # Add one-hot encoded columns to numerical/remaining features
    OH_train = pd.concat([num_train, OH_cols_train], axis=1)
    OH_test = pd.concat([num_test, OH_cols_test], axis=1)
    

    
    print(OH_train.shape,OH_test.shape)
    
    
    return OH_train,OH_test

def targetencoding(train,test,y_train,drop=False,cols=[]):
    import category_encoders as ce
    # Create the encoder itself
    cat_features = cols
    print(f'targest emcoding for features {cat_features}')
    target_enc = ce.TargetEncoder(cols=cat_features)

    

    # Fit the encoder using the categorical features and target
    target_enc.fit(train[cat_features], y_train)
    

    # Transform the features, rename the columns with _target suffix, and join to dataframe
    traintrgtenc = train.join(target_enc.transform(train[cat_features]).add_suffix('_target'))
    testtrgtenc = test.join(target_enc.transform(test[cat_features]).add_suffix('_target'))

    if drop :
        traintrgtenc = traintrgtenc.drop(cat_features, axis =1)
        testtrgtenc = testtrgtenc.drop(cat_features, axis =1)

    
    print(traintrgtenc.shape,testtrgtenc.shape)
    
    return traintrgtenc,testtrgtenc


def ordinalencoding(train,test,y_train,mapping,drop=False,cols=[]):
    import category_encoders as ce
    # Create the encoder itself
    cat_features = cols
    print(f'ordinal encoding for features {cat_features}')
    target_enc = ce.OrdinalEncoder(cols=cat_features,mapping = mapping)

    

    # Fit the encoder using the categorical features and target
    target_enc.fit(train[cat_features], y_train)
    

    # Transform the features, rename the columns with _target suffix, and join to dataframe
    traintrgtenc = train.join(target_enc.transform(train[cat_features]).add_suffix('_ordinal'))
    testtrgtenc = test.join(target_enc.transform(test[cat_features]).add_suffix('_ordinal'))

    if drop :
        traintrgtenc = traintrgtenc.drop(cat_features, axis =1)
        testtrgtenc = testtrgtenc.drop(cat_features, axis =1)

    
    print(traintrgtenc.shape,testtrgtenc.shape)
    
    return traintrgtenc,testtrgtenc

In [259]:
train_bb['DISBURSED-AMT/HIGH CREDIT'] = train_b['DISBURSED-AMT/HIGH CREDIT'].str.replace(',', '').astype('float')
test_bb['DISBURSED-AMT/HIGH CREDIT'] = test_b['DISBURSED-AMT/HIGH CREDIT'].str.replace(',', '').astype('float')

In [260]:
train_bb['CURRENT-BAL'] = train_b['CURRENT-BAL'].fillna(0)
test_bb['CURRENT-BAL'] = test_b['CURRENT-BAL'].fillna(0)
train_bb['CURRENT-BAL'] = train_b['CURRENT-BAL'].str.replace(',', '').astype('float')
test_bb['CURRENT-BAL'] = test_b['CURRENT-BAL'].str.replace(',', '').astype('float')

In [335]:

train_CURRENT = train_bb[['CURRENT-BAL','ID']].dropna()
test_CURRENT = test_bb[['CURRENT-BAL','ID']].dropna()

enc = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='kmeans')
enc.fit(train_bb['CURRENT-BAL'].dropna().values.reshape(-1,1))

train_CURRENT['CURRENT-BAL'] = enc.transform(train_CURRENT['CURRENT-BAL'].values.reshape(-1,1))
test_CURRENT['CURRENT-BAL']= enc.transform(test_CURRENT['CURRENT-BAL'].values.reshape(-1,1))

In [336]:
train_CURRENT=pd.crosstab(train_CURRENT.ID,train_CURRENT['CURRENT-BAL'],dropna= True).reset_index()
test_CURRENT=pd.crosstab(test_CURRENT.ID,test_CURRENT['CURRENT-BAL'],dropna= True).reset_index()
test_CURRENT['3.0']=0

In [346]:

train_DISBURSED = train_bb[['DISBURSED-AMT/HIGH CREDIT','ID']].dropna()
test_DISBURSED = test_bb[['DISBURSED-AMT/HIGH CREDIT','ID']].dropna()

enc = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='kmeans')
enc.fit(train_bb['DISBURSED-AMT/HIGH CREDIT'].dropna().values.reshape(-1,1))

train_DISBURSED['DISBURSED-AMT/HIGH CREDIT'] = enc.transform(train_DISBURSED['DISBURSED-AMT/HIGH CREDIT'].values.reshape(-1,1))
test_DISBURSED['DISBURSED-AMT/HIGH CREDIT']= enc.transform(test_DISBURSED['DISBURSED-AMT/HIGH CREDIT'].values.reshape(-1,1))

In [347]:
train_DISBURSED=pd.crosstab(train_DISBURSED.ID,train_DISBURSED['DISBURSED-AMT/HIGH CREDIT'],dropna= True).reset_index()
test_DISBURSED=pd.crosstab(test_DISBURSED.ID,test_DISBURSED['DISBURSED-AMT/HIGH CREDIT'],dropna= True).reset_index()
test_DISBURSED['2.0']=0

In [292]:
#discretisation baes on clusters

train_tenure = train_bb[['TENURE','ID']].dropna()
test_tenure = test_bb[['TENURE','ID']].dropna()

enc = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='kmeans')
enc.fit(train_bb['TENURE'].dropna().values.reshape(-1,1))

train_tenure['TENURE'] = enc.transform(train_tenure['TENURE'].values.reshape(-1,1))
test_tenure['TENURE']= enc.transform(test_tenure['TENURE'].values.reshape(-1,1))

In [293]:
train_tenure=pd.crosstab(train_tenure.ID,train_tenure['TENURE'],dropna= True).reset_index()
test_tenure=pd.crosstab(test_tenure.ID,test_tenure['TENURE'],dropna= True).reset_index()


In [354]:
dataframes = [train_b.ID,train_SELF_INDICATOR,train_INSTALLMENT_FREQUENCY,train_ACCOUNT_STATUS,train_OWNERSHIP_IND,
              train_CONTRIBUTOR_TYPE,train_ACCT_TYPE,train_MATCH_TYPE,train_ASSET_CLASS,train_CURRENT,train_DISBURSED,
              train_tenure]
final_df_train_beauro = reduce(lambda left, right: pd.merge(left, right, on='ID', how='left'), dataframes)

In [355]:
dataframes = [test_b.ID,test_SELF_INDICATOR,test_INSTALLMENT_FREQUENCY,test_ACCOUNT_STATUS,test_OWNERSHIP_IND,
              test_CONTRIBUTOR_TYPE,test_ACCT_TYPE,test_MATCH_TYPE,test_ASSET_CLASS,test_CURRENT,test_DISBURSED,test_tenure]
final_df_test_beauro = reduce(lambda left, right: pd.merge(left, right, on='ID', how='left'), dataframes)

In [ ]:
###### now train data 

In [475]:
train_dd = train_d.copy()
test_dd = test_d.copy()

In [476]:
train_dd.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,DisbursalAmount,...,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,State,ZiPCODE,Top-up Month
0,1,Monthly,Arrear,Closed,PDC_E,1,48,450000,275000.0,275000.0,...,4022465,1568,21946,61.11,M,49.0,35833.33,MADHYA PRADESH,464993.0,> 48 Months
1,2,Monthly,Advance,Closed,PDC,333,47,485000,350000.0,350000.0,...,4681175,1062,34802,70.00,M,23.0,666.67,MADHYA PRADESH,466001.0,No Top-up Service
2,3,Quatrly,Arrear,Active,Direct Debit,1,68,690000,519728.0,519728.0,...,25328146,1060,127335,69.77,M,39.0,45257.00,MADHYA PRADESH,462030.0,12-18 Months
3,7,Monthly,Advance,Closed,Billed,125,48,480000,400000.0,400000.0,...,13021591,1060,25094,80.92,M,24.0,20833.33,MADHYA PRADESH,473335.0,> 48 Months
4,8,Monthly,Arrear,Closed,Billed,152,44,619265,440000.0,440000.0,...,3291320,1046,21853,71.05,M,56.0,27313.67,CHATTISGARH,495442.0,36-48 Months


In [530]:
train_dd['SEX'] = train_dd['SEX'].fillna(train_dd['SEX'].mode()[0])
test_dd['SEX'] = test_dd['SEX'].fillna(test_dd['SEX'].mode()[0])

train_dd['AGE'] = train_dd['AGE'].fillna(train_dd['AGE'].mode()[0])
test_dd['AGE'] = test_dd['AGE'].fillna(test_dd['AGE'].mode()[0])

train_dd['MonthlyIncome'] = train_dd['MonthlyIncome'].fillna(train_dd['MonthlyIncome'].mode()[0])
test_dd['MonthlyIncome'] = test_dd['MonthlyIncome'].fillna(test_dd['MonthlyIncome'].mode()[0])

train_dd['ZiPCODE']  =train_dd['ZiPCODE'].fillna(method = 'ffill').astype('int') 
test_dd['ZiPCODE']  =test_dd['ZiPCODE'].fillna(method = 'ffill').astype('int') 

train_dd =train_dd.fillna(method = 'ffill') 
test_dd =test_dd.fillna(method = 'ffill') 

In [531]:
train_dd['AuthDate'] = pd.to_datetime(train_d['AuthDate'] ,errors='coerce')
test_dd['AuthDate'] = pd.to_datetime(test_d['AuthDate'] ,errors='coerce')

train_dd['DisbursalDate'] = pd.to_datetime(train_d['DisbursalDate'] ,errors='coerce')
test_dd['DisbursalDate'] = pd.to_datetime(test_d['DisbursalDate'] ,errors='coerce')

train_dd['MaturityDAte'] = pd.to_datetime(train_d['MaturityDAte'] ,errors='coerce')
test_dd['MaturityDAte'] = pd.to_datetime(test_d['MaturityDAte'] ,errors='coerce')

In [ ]:
train_dd['AuthtoMatureDays'] = train_dd['AuthDate'] - train_dd['MaturityDAte']
test_dd['AuthtoMatureDays'] = test_dd['AuthDate'] - test_dd['MaturityDAte']

train_dd['AuthtoDisDays'] = train_dd['AuthDate'] - train_dd['DisbursalDate']
test_dd['AuthtoDisDays'] = test_dd['AuthDate'] - test_dd['DisbursalDate']

In [ ]:
train_dd['AuthYear'] = train_dd['AuthDate'].dt.year 
test_dd['AuthtYear'] = test_dd['AuthDate'].dt.year 
train_dd['AuthMonth'] = train_dd['AuthDate'].dt.month 
test_dd['AuthtMonth'] = test_dd['AuthDate'].dt.month 


train_dd['MaturityYear'] = train_d['MaturityDAte'].dt.year 
test_dd['MaturityYear'] = test_d['MaturityDAte'].dt.year 

train_dd['MaturityMonth'] = train_d['MaturityDAte'].dt.month 
test_dd['MaturityMonth'] = test_d['MaturityDAte'].dt.month 

In [ ]:
train_dd = train_dd.drop(columns=['City','Area','AuthDate','DisbursalDate','MaturityDAte'])
test_dd = test_dd.drop(columns=['City','Area','AuthDate','DisbursalDate','MaturityDAte'])

In [482]:
mapping_dict = {'No Top-up Service':0, '12-18 Months':1,'18-24 Months':2, '24-30 Months':3, 
                '30-36 Months':4,'36-48 Months':5,' > 48 Months':6}

In [483]:
train_dd['Top-up Month'] = train_dd['Top-up Month'].replace(mapping_dict) 

In [497]:
mapping_dict_f= [
{'col': 'Frequency', 'mapping': {'BI-Monthly':0, 'Monthly': 1, 'Quatrly': 2,  'Half Yearly': 3}}
]

In [498]:
f1,f2 = ordinalencoding(train_dd,test_dd,train_dd['Top-up Month'],drop =True,cols=['Frequency'],mapping= mapping_dict_f)

ordinal encoding for features ['Frequency']
(128655, 24) (14745, 23)


C:\Users\HP\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [499]:
f11,f22 = onehotencoder(f1,f2,cols=['InstlmentMode','LoanStatus','SEX'])

['InstlmentMode', 'LoanStatus', 'SEX']
(128655, 27) (14745, 26)


In [500]:
f111,f222 = targetencoding(f11,f22,f11['Top-up Month'],drop=True,cols=['PaymentMode','State','BranchID','ZiPCODE','ManufacturerID']) #because branch id and area same

targest emcoding for features ['PaymentMode', 'State', 'BranchID', 'ZiPCODE', 'ManufacturerID']


C:\Users\HP\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


(128655, 27) (14745, 26)


In [ ]:
dataframes = [f111,final_df_train_beauro]
final_df_test = reduce(lambda left, right: pd.merge(left, right, on='ID', how='left'), dataframes)

In [ ]:
dataframes = [f222,final_df_test_beauro]
final_df_test = reduce(lambda left, right: pd.merge(left, right, on='ID', how='left'), dataframes)

In [ ]:
#Null value treatment